In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.corpora import Dictionary
from gensim.models.wrappers import LdaMallet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import gensim
from gensim.corpora import Dictionary
from gensim.models.wrappers import LdaMallet
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim as gensimvis
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

/Users/shtosti/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /Users/shtosti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shtosti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/shtosti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/shtosti/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [14]:
# Load the data and transform it into a DataFrame
df = pd.read_csv("/Users/shtosti/Dropbox/study/UZH/FW23/SMA/topic_modelling_DEPO/data/with_clean_keybert_bigrams.csv")
print(df.head(1))
print(type(df.cleaned_bigrams[0]))
df['cleaned_bigrams'] = df['bigram_keyphrases'].apply(lambda x: [item[0].replace(' ', '') for item in x] if x else [])
print(df.head(1))
print(type(df.cleaned_bigrams[0]))

                                          clean_text  \
0  attayyiby barackobama youtube the phrase that ...   

                                      tokenized_text  religion  spirituality  \
0  ['attayyiby', 'barackobama', 'youtube', 'phras...      True         False   

                                   bigram_keyphrases  \
0  [('away reality', 0.6107), ('reality religion'...   

                                     cleaned_bigrams  
0  ['away reality', 'reality religion', 'people r...  
<class 'str'>
                                          clean_text  \
0  attayyiby barackobama youtube the phrase that ...   

                                      tokenized_text  religion  spirituality  \
0  ['attayyiby', 'barackobama', 'youtube', 'phras...      True         False   

                                   bigram_keyphrases  \
0  [('away reality', 0.6107), ('reality religion'...   

                                     cleaned_bigrams  
0  [[, (, ', a, w, a, y, , r, e, a, l, i, t, y,

list

In [7]:
# Filter tweets containing both "spirituality" and "religion"
df = df[(df['clean_text'].str.contains('spirituality', case=False)) & 
        (df['clean_text'].str.contains('religion', case=False))]

# # Use TF-IDF vectorizer
# vectorizer = TfidfVectorizer(stop_words='english')
# X = vectorizer.fit_transform(df['clean_text'])

# # Convert the TF-IDF matrix to a Gensim-compatible format
# corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Create vocabulary and doc-term matrix
dictionary = Dictionary(df['cleaned_bigrams'])
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['cleaned_bigrams']]

# Path to the Mallet executable
mallet_path = '/Users/shtosti/Dropbox/study/UZH/FW23/SMA/topic_modelling_DEPO/Mallet/mallet-2.0.8/bin/mallet'

# Run and save the model
num_topics = 10 
lda = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=num_topics, id2word=dictionary)
os.makedirs('models', exist_ok=True)
lda.save(os.path.join('models', 'lda_religion_AND_spirituality'))


# convert Mallet LDA model to Gensim LDA model
lda_model_mallet = lda
lda_model_gensim = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model_mallet)
vis_data = gensimvis.prepare(lda_model_gensim, doc_term_matrix, dictionary)

# save to HTML
html_dir = 'visualizations'
os.makedirs(html_dir, exist_ok=True)
html_filename = os.path.join(html_dir, f'lda_religion_AND_spirituality_{num_topics}_topics.html')
pyLDAvis.save_html(vis_data, html_filename)

TypeError: doc2bow expects an array of unicode tokens on input, not a single string